# Tracer Advection Stencil

### Start a parallel cluster

In [ ]:
import ipyparallel as ipp

cluster = ipp.Cluster(engines='mpi', n=6).start_and_connect_sync()

Starting 6 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>


  0%|          | 0/6 [00:00<?, ?engine/s]

In [ ]:
%autopx

### Input parameters

In [ ]:
%%capture
from mpi4py import MPI
import copy as cp
import functions as func
import numpy as np
import os
#import importlib
#importlib.reload(func)


mpi_comm = MPI.COMM_WORLD

backend = 'numpy'
layout = (1, 1)
nx, ny, nz, nhalo = 100, 100, 79, 3        # number of points in x, y, z dimensions, halos
nDays = 12
timestep = 900.                            # time step in seconds

test_case = 'b'                            # a: fortran test_case1; b; less dispersive
print_advectionProgress = True             # True: print timestep differences on rank 0
plot_output = True                         # True: plot output 
figure_everyNsteps = int(3600/timestep*6)  # how often to plot a figure (in hours)
write_initialCondition = True              # True: write out initial condition and coordinate system

# initial condition adjustables
pressure_base = 10                         # pressure thickness of layer in Pa
smoke_base = 1                             # maximum smoke amount
smoke_center_tile = 0                      # rank of tile on which to center the gaussian tracer
density = 1                                # density of air for mass flux

# finite volume transport options
grid_type = 0                              # cubed-sphere
hord = 6                                   # horizontal advection order

######-------- this can be left alone --------######
# frequency of storing and plotting tracer state
nSeconds = nDays * 86400 + timestep*5      
nSteps_advection = int(nSeconds/timestep + 1)

tracer_archive_frequency = 1               # store tracer state after every time step

pwd = os.getcwd()
nx_effective, ny_effective, nz_effective = nx + 1, ny + 1, nz + 1

if figure_everyNsteps > nSteps_advection:
    figure_everyNsteps = nSteps_advection

## Prepare state for advection

In [ ]:
%%capture
dimensions, units, origins, rank = func.define_dimensionsUnitsOrigins(nx_effective, ny_effective, nz_effective, nhalo, mpi_comm)
smoke_dict = {'center_tile': smoke_center_tile, 'rank': rank, 'smoke_base': smoke_base}
pressure_dict = {'pressure_base': pressure_base}
fvt_dict = {'grid_type': grid_type, 'hord': hord}

configuration = func.configure_domain(layout, mpi_comm, dimensions, backend=backend)
if test_case == 'a':
    initialState = func.create_initialState_testCase1a(configuration['grid_data'], dimensions, units, origins, 
                                                       backend, smoke_dict, pressure_dict)
elif test_case == 'b':
    initialState = func.create_initialState_testCase1b(configuration['grid_data'], dimensions, units, origins, 
                                                       backend, smoke_dict, pressure_dict)

tracAdv_data, tracAdv = func.prepare_everythingForAdvection(configuration, initialState['uC'], initialState['vC'], 
                                                            initialState['delp'], {'smoke': initialState['smoke']}, 
                                                            density, dimensions, units, origins, backend, 
                                                            fvt_dict, dt_acoustic=timestep)
                                                            
tracAdv_dataInit = cp.deepcopy(tracAdv_data)


if write_initialCondition:
    saveDir = pwd + '/data/'
    fOut = saveDir + 'initialState_%sx%s_%s.nc' % (dimensions['nx']-1, dimensions['ny']-1, test_case)
            
    func.write_initialCondition_toFile(fOut, initialState, dimensions, units, origins, backend, configuration, rank)


## Advection loop

In [ ]:
tracer_archive = [cp.deepcopy(tracAdv_data['tracers']['smoke'])]
timestep_archive = [0]
for step in range(nSteps_advection):    
    if (rank == 0) and (step+1)%100 == 0 and print_advectionProgress:
        print('Step: %d' % (step+1))
    #tracAdv_data = func.run_advectionStepWithReset(tracAdv_dataInit, tracAdv_data, tracAdv, timestep, mpi_rank=rank)
    tracAdv_data = func.run_advectionStepWithReset(tracAdv_dataInit, tracAdv_data, tracAdv, timestep)

    if (step+1)%tracer_archive_frequency == 0:
        tracer_archive.append(cp.deepcopy(tracAdv_data['tracers']['smoke']))
        timestep_archive.append(step+1)

## Plot output

In [ ]:
#%%capture
if plot_output:
    lon, lat = func.get_lonLat_edges(configuration['grid_data'], dimensions, units, origins, backend)
    lon_global = configuration['communicator'].gather(lon)
    lat_global = configuration['communicator'].gather(lat)
    smoke_global = []
    for step in range(len(timestep_archive)):
        smoke_global.append(configuration['communicator'].gather(tracer_archive[step]))

    if rank == 0:
        saveDir = pwd + '/figs/'
        
        if not os.path.isdir(saveDir):
            os.mkdir(saveDir)

        smoke_global = np.squeeze(np.stack(smoke_global))
        for plotStep in range(0, len(timestep_archive), figure_everyNsteps):
            print('Plotting time step %s / %s' % (timestep_archive[plotStep], nSteps_advection))
            fOut = saveDir + 'advection_step_' + test_case
            fOut_diff = saveDir + 'advection_step_diff_' + test_case
            func.plot_projection_field(lon_global.data, lat_global.data, smoke_global[plotStep], 
                                        cmap='viridis', vmin=0, vmax=1, units='kg', 
                                        title='Tracer after %.2f days - case %s' % 
                                                ((timestep_archive[plotStep] * timestep)/60/60/24, test_case), 
                                        fSave=fOut+'{:06.2f}'.format((timestep_archive[plotStep] * timestep)/60/60) + '.png')
            func.plot_projection_field(lon_global.data, lat_global.data, smoke_global[plotStep] - smoke_global[0], 
                                        cmap='bwr', vmin=-1, vmax=1, units='kg', 
                                        title='Tracer difference from initial state after %.2f days - case %s' % 
                                                ((timestep_archive[plotStep] * timestep)/60/60/24, test_case), 
                                        fSave=fOut_diff+'{:06.2f}'.format((timestep_archive[plotStep] * timestep)/60/60) + '.png')

In [ ]:
#%autopx

In [ ]:
#cluster.shutdown()